In [ ]:
%cd "~/moses-incons-pen-xp"
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import scipy
import math
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
import math
from sklearn.model_selection import train_test_split
from datetime import datetime
from notebooks.manifold_reg.util import *

In [ ]:
X, beta, y_log = generate_log_data(200, 10, tf_on=4, val_tf=[5, 5, 5, 5], n=200)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.5, stratify=y_log)

In [ ]:
assoc_mat = get_assoc_mat(200, 10)

In [ ]:
n_iter = 15
l1_vals = np.logspace(-2, 0, n_iter)
# l1_vals = np.concatenate([np.array([0]), l1_vals])
l2_vals = np.logspace(-5, 1, n_iter)
# l2_vals = np.concatenate([np.array([0]), l2_vals])
gammas = np.logspace(-2, 3, 10)

In [ ]:
gammas

Run Logistic regression with $\Vert\beta\Vert^{2}$ and $\mathbf{}{f}^{T}\mathbf{}{L}\mathbf{}{f}$ using **gradient descent**

In [ ]:
from notebooks.manifold_reg.grad_util import run_logisitic_reg_exp, build_table_from_log

train_errs_cv_0, test_errs_0, train_ll_pen_0, test_ll_pen_0, \
    train_l1_pen_0, test_l1_pen_0, train_l2_pen_0, test_l2_pen_0, min_l1_vals_0, min_l2_vals_0, _ = run_logisitic_reg_exp(X_train, X_test, y_train, y_test, assoc_mat, gammas, l1_vals, l2_vals, l2_norm=True)

res_1_df = build_table_from_log(gammas, train_errs_cv_0, test_errs_0, train_ll_pen_0, test_ll_pen_0,
                                train_l1_pen_0, test_l1_pen_0, min_l1_vals_0, train_l2_pen_0, test_l2_pen_0, min_l2_vals_0)

res_1_df

Run Logistic regression with $\Vert\beta\Vert^{1}$ and $\mathbf{}{f}^{T}\mathbf{}{L}\mathbf{}{f}$ using **gradient descent**

In [ ]:
from notebooks.manifold_reg.grad_util import run_logisitic_reg_exp, build_table_from_log

# l1_vals = np.logspace(-2, 0, n_iter)
# l1_vals = np.concatenate([np.array([0]), l1_vals])
# l2_vals = np.logspace(-4, -2, n_iter)

train_errs_cv, test_errs, train_ll_pen, test_ll_pen, \
    train_l1_pen, test_l1_pen, train_l2_pen, test_l2_pen, min_l1_vals, min_l2_vals, _ = run_logisitic_reg_exp(X_train, X_test, y_train, y_test, assoc_mat, gammas, l1_vals, l2_vals, l2_norm=False)

res_2_df = build_table_from_log(gammas, train_errs_cv, test_errs, train_ll_pen, test_ll_pen,
                                train_l1_pen, test_l1_pen, min_l1_vals, train_l2_pen, test_l2_pen, min_l2_vals)

res_2_df

Focus on $\gamma$ values between 1 and 10 with $\Vert\beta\Vert^{2}$ norm and $\mathbf{}{f}^{T}\mathbf{}{L}\mathbf{}{f}$

In [ ]:
gammas_2 = np.logspace(0, 1, 5)

train_errs_cv, test_errs, train_ll_pen, test_ll_pen, \
    train_l1_pen, test_l1_pen, train_l2_pen, test_l2_pen, min_l1_vals, min_l2_vals = run_logisitic_reg_exp(X_train, X_test, y_train, y_test, assoc_mat, gammas_2, l1_vals, l2_vals)

res_3_df = build_table_from_log(gammas_2, train_errs_cv, test_errs, train_ll_pen, test_ll_pen,
                                train_l1_pen, test_l1_pen, min_l1_vals, train_l2_pen, test_l2_pen, min_l2_vals)

res_3_df

Focus on $\gamma$ values between 1 and 10 with $\Vert\beta\Vert^{1}$ norm and $\mathbf{}{f}^{T}\mathbf{}{L}\mathbf{}{f}$

In [ ]:
train_errs_cv, test_errs, train_ll_pen, test_ll_pen, \
    train_l1_pen, test_l1_pen, train_l2_pen, test_l2_pen, min_l1_vals, min_l2_vals = run_logisitic_reg_exp(X_train, X_test, y_train, y_test, assoc_mat, gammas_2, l1_vals, l2_vals, l2_norm=False)

res_4_df = build_table_from_log(gammas_2, train_errs_cv, test_errs, train_ll_pen, test_ll_pen,
                                train_l1_pen, test_l1_pen, min_l1_vals, train_l2_pen, test_l2_pen, min_l2_vals)

res_4_df

Chech for different values of $\lambda_{1}$ by setting $\lambda_{2}$ to 0

In [ ]:
train_errs_cv, test_errs, train_ll_pen, test_ll_pen, \
    train_l1_pen, test_l1_pen, train_l2_pen, test_l2_pen, min_l1_vals, min_l2_vals = run_logisitic_reg_exp(X_train, X_test, y_train, y_test, assoc_mat, np.array([1.0]), l1_vals, np.zeros(n_iter), l2_norm=False)

res_5_df = build_table_from_log(np.array([1.0]), train_errs_cv, test_errs, train_ll_pen, test_ll_pen,
                                train_l1_pen, test_l1_pen, min_l1_vals, train_l2_pen, test_l2_pen, min_l2_vals)

res_5_df

Sample repeatedly from the distribution and compare with and without penalty. For penalty version, use $\gamma$ = 1, $\Vert\beta\Vert^{1}$ and $\mathbf{}{f}^{T}\mathbf{}{L}\mathbf{}{f}$ penalty

In [ ]:
from notebooks.manifold_reg.grad_util import *
import warnings
import traceback

warnings.filterwarnings('ignore')
results_1 = []
results_2 = []

n_sim = 10
beta_vals_1 = np.zeros((n_sim, 1, X_train.shape[1]))
beta_vals_2 = np.zeros((n_sim, 1, X_train.shape[1]))

gamma = np.array([1.0])
for i in range(n_sim):
    try:
        #log_msg(f"[{datetime.now()}] - Sim {i + 1}...")
        X_exp, beta_exp, y_log_exp = generate_log_data(200, 10, tf_on=4, val_tf=[5, 5, 5, 5], n=200)
        X_train_exp, X_test_exp, y_train_exp, y_test_exp = train_test_split(X_exp, y_log_exp, test_size=0.5, stratify=y_log_exp)

        train_errs_cv_1, test_errs_1, train_ll_pen_1, test_ll_pen_1, \
        train_l1_pen_1, test_l1_pen_1, train_l2_pen_1, test_l2_pen_1, min_l1_vals_1, min_l2_vals_1, beta_vals_1[i] = run_logisitic_reg_exp(X_train_exp, X_test_exp, y_train_exp, y_test_exp, assoc_mat, gamma, l1_vals, l2_vals, l2_norm=False)

        res_pen_df = build_table_from_log(gamma, train_errs_cv_1, test_errs_1, train_ll_pen_1, test_ll_pen_1,
                                        train_l1_pen_1, test_l1_pen_1, min_l1_vals_1, train_l2_pen_1, test_l2_pen_1, min_l2_vals_1)

        train_errs_cv_2, test_errs_2, train_ll_pen_2, test_ll_pen_2, \
        train_l1_pen_2, test_l1_pen_2, min_l1_vals_2, beta_vals_2[i] = run_logisitic_reg_exp(X_train_exp, X_test_exp, y_train_exp, y_test_exp, assoc_mat, gamma, l1_vals, None, l2_norm=False)

        res_no_pen_df = build_table_from_log(gamma, train_errs_cv_2, test_errs_2, train_ll_pen_2, test_ll_pen_2,
                                        train_l1_pen_2, test_l1_pen_2, min_l1_vals_2, lap=False)


        results_1.append(res_pen_df)
        results_2.append(res_no_pen_df)
    except Exception as e:
        #log_msg("Oh, no! An error occurred. Here is the error message")
        #log_msg(traceback.format_exc())

#log_msg("Done")

In [ ]:
#Fix argument mismatch in column names when building the dataframe
# results_1_pen_df["l1_train_cv"] = results_1_pen_df_cp["l2_pen_test"]
# results_1_pen_df["l2_pen_train_cv"] = results_1_pen_df_cp["l1_train_cv"]
# results_1_pen_df["l2_pen_test"] = results_1_pen_df_cp["l2_pen_train_cv"]

results_1_pen_df = pd.concat(results_1, axis=0)
results_1_pen_df

In [ ]:
results_1_pen_df.mean(axis=0)

In [ ]:
results_2_pen_df = pd.concat(results_2, axis=0)
results_2_pen_df

In [ ]:
results_2_pen_df.mean(axis=0)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

r = np.arange(1, 11)
ax[0].plot(r, results_1_pen_df["min_train_err_cv"], color="blue", label="with pen")
ax[0].plot(r, results_2_pen_df["min_train_err_cv"], color="red", label="w/o pen")
ax[0].set_xlabel(r"run", fontsize=16)
ax[0].set_ylabel(r"log_loss", fontsize=16)
ax[0].set_title("Train Error")
ax[0].legend()

ax[1].plot(r, results_1_pen_df["test_err"], color="blue", label="with pen")
ax[1].plot(r, results_2_pen_df["test_err"], color="red", label="w/o pen")
ax[1].set_xlabel(r"run", fontsize=16)
ax[1].set_ylabel(r"log_loss", fontsize=16)
ax[1].set_title("Test Error")
ax[1].legend()

In [ ]:
train_errs_cv, test_errs, train_ll_pen, test_ll_pen, \
    train_l1_pen, test_l1_pen, train_l2_pen, test_l2_pen, min_l1_vals, min_l2_vals, beta_val_3 = run_logisitic_reg_exp(X_train, X_test, y_train, y_test, assoc_mat, np.array([1.0]), l1_vals, np.logspace(0, 3, n_iter), l2_norm=False)

res_6_df = build_table_from_log(gamma, train_errs_cv, test_errs, train_ll_pen, test_ll_pen,
                                        train_l1_pen, test_l1_pen, min_l1_vals, train_l2_pen, test_l2_pen, min_l2_vals)

In [ ]:
perfect_ll_train = np.zeros(100)
perfect_ll_test = np.zeros(100)

for i in range(100):
    X_exp, beta_exp, y_log_exp = generate_log_data(200, 10, tf_on=4, val_tf=[5, 5, 5, 5], n=200)

    X_train_exp, X_test_exp, y_train_exp, y_test_exp = train_test_split(X_exp, y_log_exp, test_size=0.5, shuffle=True)

    p_train = sigmoid(X_train_exp @ beta_exp)
    p_test = sigmoid(X_test_exp @ beta_exp)

    perfect_ll_train[i] = log_loss(y_train_exp, p_train)
    perfect_ll_test[i] = log_loss(y_test_exp, p_test)

In [ ]:
plt.plot(np.arange(0, 100), perfect_ll_train, color="blue")
plt.plot(np.arange(0, 100), perfect_ll_test, color="red")

In [ ]:
np.mean(perfect_ll_train)

In [ ]:
np.mean(perfect_ll_test)